In [35]:
# Step 1: Define the libraries, settings and dependencies

import json
from sklearn.feature_extraction.text import TfidfVectorizer 
import re
import ftfy as ft
import time
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)



In [36]:
# Step 2: Import the files and load them up 


t=open(r"C:\dev\personal\Web Scraping\projects\tesco\tesco_072720.json")
a=open(r"C:\dev\personal\Web Scraping\projects\asda\asda_072720.json")
s=open(r"C:\dev\personal\Web Scraping\projects\sainsburys\sainsburys_072720.json")
w=open(r"C:\dev\personal\Web Scraping\projects\waitrose\waitrose_072720.json")
m=open(r"C:\dev\personal\Web Scraping\projects\morrisons\morrisons_072720.json")
tesco_price=json.load(t)
# print(tesco_price)
ctr=1
tesco_prod_name=[]
for i in tesco_price:
    tesco_prod_name.insert(ctr,i['tesco_prod_name'])
    ctr+=1

#print(tesco_prod_name)
t.close()

asda_price=json.load(a)
# print(asda_price)
ctr=1
asda_prod_name=[]
for i in asda_price:
    asda_prod_name.insert(ctr,i['asda_prod_name'])
    ctr+=1

#print(asda_prod_name)
a.close()

morrisons_price=json.load(m)
# print(morrisons_price)
ctr=1
morrisons_prod_name=[]
for i in morrisons_price:
    morrisons_prod_name.insert(ctr,i['morrisons_prod_name'])
    ctr+=1

#print(morrisons_prod_name)
m.close()

waitrose_price=json.load(w)
# print(waitrose_price)
ctr=1
waitrose_prod_name=[]
for i in waitrose_price:
    waitrose_prod_name.insert(ctr,i['waitrose_prod_name'])
    ctr+=1

#print(waitrose_prod_name)
w.close()

sainsburys_price=json.load(s)
# print(sainsburys_price)
ctr=1
sainsburys_prod_name=[]
for i in sainsburys_price:
    sainsburys_prod_name.insert(ctr,i['sainsburys_prod_name'])
    ctr+=1

#print(sainsburys_prod_name)
s.close()



In [37]:
# Step 3: Identify the unique names in the product lists 

def uniq(list1):
    list_set=set(list1)
    unique_list=list(list_set)
    return unique_list


tesco_prod_name_uniq=uniq(tesco_prod_name)
asda_prod_name_uniq=uniq(asda_prod_name)
morrisons_prod_name_uniq=uniq(morrisons_prod_name)
sainsburys_prod_name_uniq=uniq(sainsburys_prod_name)
waitrose_prod_name_uniq=uniq(waitrose_prod_name)
prim_vec_count=len(tesco_prod_name_uniq)
trg_vec_count=len(morrisons_prod_name_uniq)
test_vector=np.array(tesco_prod_name_uniq+morrisons_prod_name_uniq)


print(len(test_vector))
print(prim_vec_count)

print("Shape of tesco: pre deduping- ", len(tesco_prod_name), "; post deduping- ", len(tesco_prod_name_uniq))
print("Shape of asda: pre deduping- ", len(asda_prod_name), "; post deduping- ", len(asda_prod_name_uniq))
print("Shape of morrisons: pre deduping- ", len(morrisons_prod_name), "; post deduping- ", len(morrisons_prod_name_uniq))
print("Shape of sainsburys: pre deduping- ", len(sainsburys_prod_name), "; post deduping- ", len(sainsburys_prod_name_uniq))
print("Shape of waitrose: pre deduping- ", len(waitrose_prod_name), "; post deduping- ", len(waitrose_prod_name_uniq))




544
204
Shape of tesco: pre deduping-  204 ; post deduping-  204
Shape of asda: pre deduping-  248 ; post deduping-  61
Shape of morrisons: pre deduping-  340 ; post deduping-  340
Shape of sainsburys: pre deduping-  179 ; post deduping-  179
Shape of waitrose: pre deduping-  135 ; post deduping-  135


In [34]:

# Step 4: Data cleaning

frtoeng = "".maketrans("àâçéèêîôùû", "aaceeeiouu")

def cleanup(string):
    string = ft.fix_text(string) # fix text encoding issues
    string = string.translate(frtoeng)
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower() #make lower case
    chars_to_remove = [")","(",".","|","[","]","{","}","'","`","/","~"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string) #remove the list of chars defined above
    string = string.replace('&', '')
    string = string.replace(' ml', 'ml')
    string = string.replace('and', '')
    string = string.replace('ml', '')
    string = string.replace('Ml', 'ml')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.replace('+', ' ')
    #string="".join(sorted(string))
    #string = string.replace('shampoo', '') (doesnt work as most of the conditioners come into play which they shouldnt)
    #string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single space
    # string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    return string

def ngrams(string, n=3):
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


f= lambda x: cleanup(x)
tesco_prod_name_uniq_cleaned= np.array([f(xi) for xi in tesco_prod_name_uniq])
morrisons_prod_name_uniq_cleaned= np.array([f(xi) for xi in morrisons_prod_name_uniq])
waitrose_prod_name_uniq_cleaned= np.array([f(xi) for xi in waitrose_prod_name_uniq])
sainsburys_prod_name_uniq_cleaned= np.array([f(xi) for xi in sainsburys_prod_name_uniq])
asda_prod_name_uniq_cleaned= np.array([f(xi) for xi in sainsburys_prod_name_uniq])


#Test Cases
## Case I
print(cleanup("L'OrÃƒÂ©al Elvive Extraordinary Oil Shampoo Dry Hair  250ml"))
print("L'OrÃƒÂ©al Elvive Extraordinary Oil Shampoo Dry Hair  250ml"["L'OrÃƒÂ©al Elvive Extraordinary Oil Shampoo Dry Hair  250ml".rstrip().rfind(" "):-2])
j="L'OrÃƒÂ©al Elvive Extraordinary Oil Shampoo Dry Hair  250ml"["L'OrÃƒÂ©al Elvive Extraordinary Oil Shampoo Dry Hair  250ml".rstrip().rfind(" "):-2]

## Case II

print("".join(sorted(cleanup("Herbal Essences Bio. Renew Argan Shampoo 400Ml"))))
print("".join(sorted(cleanup("Herbal Essences Bio Renew Shampoo Coconut Milk Hydrate 400ml"))))
print("".join(sorted(cleanup("Herbal Essences Bio Renew Shampoo Argan Oil of Morocco 400ml")))) #ideally this one should be selected as the right match


## Case III


# print("".join(sorted(cleanup("Alberto Balsam Apple Shampoo 350Ml"))))

# k=[x for x in morrisons_prod_name if x.find("Alberto Balsam")!=-1]
# l=np.array(uniq(k))
# print(l)
# ratio = process.extract(cleanup("Alberto Balsam Apple Shampoo 350Ml"), morrisons_prod_name_uniq_cleaned, limit=1, scorer=fuzz.partial_token_set_ratio)
# print(ratio[0][0])
# res=np.where(morrisons_prod_name_uniq_cleaned==ratio[0][0])
# print(morrisons_prod_name_uniq[res[0][0]])
# k=np.array([x for x in morrisons_prod_name if x.find(j)!=-1])
# print(k)


loreal elvive extraordinary oil shampoo dry hair 250
 250
      004aaaabbceeeeeeghhilmnnnoooprrrssssw
        004aaabbcccdeeeeeeehhhiikllmmnnnoooooprrrssssttuwy
         004aaaabbccceeeeeefghhiillmmnnnooooooooprrrrssssw
    035aaaaabbeehlllmmoooppprsst
['Alberto Balsam Energize Shampoo 250ml'
 'Alberto Balsam Juicy Green Apple Shampoo  350ml'
 'Alberto Balsam Restore Shampoo 250ml'
 'Alberto Balsam Coconut & Lychee Conditioner  350ml'
 'Alberto Balsam Energize Conditioner 250ml'
 'Alberto Balsam Juicy Green Apple Conditioner  350ml'
 'Alberto Balsam Raspberry Shampoo 350ml'
 'Alberto Balsam Tea Tree Tingle Shampoo  350ml'
 'Alberto Balsam Coconut & Lychee Nourishing Shampoo 350ml'
 'Alberto Balsam Restore Conditioner 250ml'
 'Alberto Balsam Tea Tree Tingle Conditioner  350ml'
 'Alberto Balsam Raspberry Conditioner 350ml']


NameError: name 'process' is not defined

In [30]:
# Step 5: Define Cosine Similarity Function and a Function to return matches

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))


def get_matches_df(sparse_matrix, name_vector, top=0):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    #print(sparsecols.size)
    #print(sparserows.size)
    if top!= 0:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    Tesco = np.empty(1, dtype=object) # returns a matrix of given shape
    right_side = np.empty(1, dtype=object) # returns a matrix of given shape
    similairity = np.zeros(1) # returns a matrix of given shape with zeros
    i=0
    for index in range(0, nr_matches):
        if (sparserows[index] < 1) and (sparsecols[index] >= 1) and (sparserows[index] != sparsecols[index]):
            Tesco[i] = name_vector[sparserows[index]]
            right_side[i] = name_vector[sparsecols[index]]
            similairity[i] = sparse_matrix.data[index]
            i=i+1

    return pd.DataFrame({'Tesco': Tesco,
                          'right_side': right_side,
                           'similairity': similairity})



In [33]:
# Step 6: Results section
def process_all(srch_dset, outdset, prim_dset=tesco_prod_name_uniq):
    ctr=0
    for i in prim_dset:
        print(i)
        i1=cleanup(i)
        print(i1)
        j=i1[i1.rstrip().rfind(" "):]
        print(j)
        k=[x for x in srch_dset if x.find(j)!=-1]
        if i1.find("shampoo")!=-1:
            k=[x for x in k if x.find("hampoo")!=-1]
        # print(k)
        l1=np.array([i]+k)
        l=np.array([i1]+[f(xi) for xi in k])
        # print(l)
        if l.size==1:
            primdset = np.empty([1], dtype=object) # returns a matrix of given shape
            right_side = np.empty([1], dtype=object) # returns a matrix of given shape
            similairity = np.zeros(1) # returns a matrix of given shape with zeros
            primdset[0] = i
            right_side[0] = ""
            similairity[0] = 0
            matches_df=pd.DataFrame({'Tesco': primdset,
                        'right_side': right_side,
                        'similairity': similairity})
        else:
            T = vectorizer.fit_transform(l)
            matches = awesome_cossim_top(T, T.transpose(), 2, 0.01)
            print(matches.shape)
            matches_df = get_matches_df(matches, l1)

        if ctr==0 :
            srchmatch=matches_df
        else:
            srchmatch=srchmatch.append(matches_df)
        ctr=ctr+1
    print(outdset)
    srchmatch.to_csv(f'C:/dev/personal/Web Scraping/projects/Data Processing/processed/{outdset}.csv')

process_all(srch_dset=morrisons_prod_name_uniq, outdset='tesco_morrisons',prim_dset=tesco_prod_name_uniq)

Pantene Smooth & Sleek Shampoo & Conditioner 3In1 450Ml
pantene smooth sleek shampoo conditioner 3in1 450
 450
(5, 5)
L'oreal Elvive Clay Oily Roots Shampoo 500Ml
loreal elvive clay oily roots shampoo 500
 500
(10, 10)
Batiste Heavenly Volume Dry Shampoo 200Ml
batiste heavenly volume dry shampoo 200
 200
(6, 6)
Aussie Beach Mate Shampoo Limited Edition 300Ml
aussie beach mate shampoo limited edition 300
 300
(15, 15)
John Frieda Luxurious Volume Colour Care Shampoo 250Ml
john frieda luxurious volume colour care shampoo 250
 250
(34, 34)
Aussie Miracle Moist Shampoo 300Ml
aussie miracle moist shampoo 300
 300
(15, 15)
Hask Keratin Protein Shampoo 355Ml
hask keratin protein shampoo 355
 355
Aussie Colour Mate Shampoo 300Ml
aussie colour mate shampoo 300
 300
(15, 15)
Neutrogena T/Gel 2 In 1 Dandruff Shampoo 250Ml
neutrogena tgel 2 in 1 druff shampoo 250
 250
(34, 34)
Ogx Keratin Oil Anti-Breakage Shampoo 385Ml
ogx keratin oil anti breakage shampoo 385
 385
(9, 9)
Maui Moisture Shea Butte

In [ ]:
t1 = time.time()
matches = awesome_cossim_top(T, T.transpose(), 5, 0.5)
t = time.time()-t1
print("SELFTIMED:", t)
print("Shape of matches")
print(matches.shape)
print(matches[3])

In [ ]:


matches_df = get_matches_df(matches, test_vector)
matches_df.dropna(subset = ["Tesco"], inplace=True)
print(matches_df.shape)
# print(matches_df.head(100))
matches_df= matches_df.groupby(["Tesco"]).apply(lambda x: x.sort_values(["similairity"], ascending = False)).reset_index(drop=True)
print(matches_df)
matches_df=matches_df.groupby(["Tesco"]).head(1)
matches_df.to_csv('C:/Users/sujithk/Desktop/file_name.csv')
# print(matches_df.sort_values(['similairity'], ascending=False))